<a href="https://colab.research.google.com/github/cange95/NLP/blob/main/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  aitextgen — Train a Custom GPT-2 Model + Tokenizer w/ GPU

by [Max Woolf](https://minimaxir.com)

*Last updated: May 16th, 2021 (aitextgen v0.5.2)*

Train a custom GPT-2 model **for free on a GPU using Colaboratory** using `aitextgen`!

It's recommended to only create a model from scratch if you really need to do so; otherwise, [finetuning 124M](https://colab.research.google.com/drive/15qBZx5y9rdaQSyWpsreMDnTiZ5IlN0zD?usp=sharing) may give you better results.

For more about `aitextgen`, you can visit [this GitHub repository](https://github.com/minimaxir/aitextgen) or [read the documentation](https://docs.aitextgen.io/).


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Run the cells below:


In [1]:
!pip install -q aitextgen

import logging
logging.basicConfig(
        format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO
    )

from aitextgen import aitextgen
from aitextgen.colab import mount_gdrive, copy_file_from_gdrive
from aitextgen.TokenDataset import TokenDataset, merge_datasets
from aitextgen.utils import build_gpt2_config
from aitextgen.tokenizers import train_tokenizer

     |████████████████████████████████| 572 kB 4.4 MB/s 
     |████████████████████████████████| 3.1 MB 30.6 MB/s 
     |████████████████████████████████| 87 kB 6.3 MB/s 
     |████████████████████████████████| 523 kB 46.3 MB/s 
     |████████████████████████████████| 596 kB 45.5 MB/s 
     |████████████████████████████████| 329 kB 48.8 MB/s 
     |████████████████████████████████| 829 kB 39.0 MB/s 
     |████████████████████████████████| 132 kB 45.3 MB/s 
     |████████████████████████████████| 1.1 MB 37.2 MB/s 
     |████████████████████████████████| 895 kB 31.5 MB/s 
     |████████████████████████████████| 3.3 MB 36.9 MB/s 
     |████████████████████████████████| 59 kB 5.5 MB/s 
     |████████████████████████████████| 271 kB 49.5 MB/s 
     |████████████████████████████████| 192 kB 49.5 MB/s 
     |████████████████████████████████| 160 kB 48.1 MB/s 


## GPU

Colaboratory uses a Nvidia P4, an Nvidia T4, or an Nvidia P100 GPU. For finetuning GPT-2 124M, any of these GPUs will be fine, but for text generation, a T4 or a P100 is ideal since they have more VRAM.

You can verify which GPU is active by running the cell below. If you want to try for a different GPU, go to **Runtime -> Factory Reset Runtime**.

In [2]:
!nvidia-smi

Fri Nov 26 13:19:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [3]:
mount_gdrive()

Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/w3wvHhR.png)

Upload **any smaller text file** (for example, [a text file of Shakespeare plays](https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt)) and update the file name in the cell below, then run the cell.

In [23]:
file_name = "_chat.txt"

In [ ]:
###Filter, if you want
with open(file_name, "r") as f:
    lines = f.readlines()
    
with open(file_name, "w") as f:
    for line in lines:
        if " omitted" not in line:
            f.write(line)
        else:
          print(line)

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
copy_file_from_gdrive(file_name)

## Training the Tokenizer

Now we can train a Byte-Pair Encoding tokenizer on the dataset we just downloaded. The `train_tokenizer()` function wraps the training method for the `tokenizer` package from Huggingface.

After the training is completed, this will save one file: **aitextgen.tokenizer.json**, which is needed to rebuild the tokenizer.

In [11]:
train_tokenizer(file_name)

In [15]:
from transformers import PreTrainedTokenizerFast

fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file="aitextgen.tokenizer.json")
print(fast_tokenizer("Ciao culo"))
print(fast_tokenizer("Ciao baby"))

{'input_ids': [35, 316, 79, 935], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}
{'input_ids': [35, 316, 79, 380], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}


## Specify a Model Configuration

You can use `build_gpt2_config` to specify a model configuration. You most likely will want to adjust `max_length` (context window size) and `n_embd` (embedding size).

The config used here is the one used to build a [demo Reddit](https://github.com/minimaxir/aitextgen/blob/master/notebooks/reddit_demo.ipynb) model.

In [20]:
config = build_gpt2_config(vocab_size=1000, max_length=64, dropout=0.0, n_embd=128, n_layer=8, n_head=8)
config

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.0,
  "bos_token_id": 0,
  "embd_pdrop": 0.0,
  "eos_token_id": 0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 64,
  "n_embd": 128,
  "n_head": 8,
  "n_inner": null,
  "n_layer": 8,
  "n_positions": 64,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.0,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.0,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.12.5",
  "use_cache": true,
  "vocab_size": 1000
}

## Instantiating Your Custom GPT-2 Model

Pass all the information to `aitextgen()` and you're good to go!

In [21]:
ai = aitextgen(config=config,
               tokenizer_file="aitextgen.tokenizer.json",
               to_gpu=True)

11/26/2021 13:47:41 — INFO — aitextgen — Constructing model from provided config.
11/26/2021 13:47:41 — INFO — aitextgen — GPT2 loaded with 1M parameters.
11/26/2021 13:47:41 — INFO — aitextgen — Using a custom tokenizer.


Generated output from it will be effectively random, for now.

In [18]:
ai.generate(5)

�F�q ha��ggibb carinooiannoorna� doveani34ono P appnessand anche53 fa��Q Traci
19neene 15 s&&FF �ui te 00gno gtriìnimente Poihega Babyyy omidano all desie nel
na�ri� amo Siui�� bello pren Grazie�na y09ìani‎[20 ominini Ftuèp�cco
gabbe‎[ooooo, ne �zalleonahahaha nel�� lo lo Corsi Corsi 13�#uto37 Lo � min haique� � ch
w app� InL�bbesì tross co Noneegg44 chia Di�ne così!he �siz16p


## Train GPT-2

The next cell will start the actual training of GPT-2 in aitextgen. It runs for `num_steps`, and a progress bar will appear to show training progress, current loss (the lower the better the model), and average loss (to give a sense on loss trajectory).

The model will be saved every `save_every` steps in `trained_model` by default, and when training completes. If you mounted your Google Drive, the model will _also_ be saved there in a unique folder.

The training might time out after 4ish hours; if you did not mount to Google Drive, make sure you end training and save the results so you don't lose them! (if this happens frequently, you may want to consider using [Colab Pro](https://colab.research.google.com/signup))

Important parameters for `train()`:

- **`line_by_line`**: Set this to `True` if the input text file is a single-column CSV, with one record per row. aitextgen will automatically process it optimally.
- **`from_cache`**: If you compressed your dataset locally (as noted in the previous section) and are using that cache file, set this to `True`.
- **`num_steps`**: Number of steps to train the model for.
- **`generate_every`**: Interval of steps to generate example text from the model; good for qualitatively validating training.
- **`save_every`**: Interval of steps to save the model: the model will be saved in the VM to `/trained_model`.
- **`save_gdrive`**: Set this to `True` to copy the model to a unique folder in your Google Drive, if you have mounted it in the earlier cells
- **`batch_size`**: Batch size of the model training; setting it too high will cause the GPU to go OOM. _Unlike finetuning, since you are using a small model, you can massively increase the batch size to normalize the training_.
- **`fp16`**: Enables half-precision training for faster/more memory-efficient training. Only works on a T4 or V100 GPU.

Here are other important parameters for `train()` that are useful but you likely do not need to change.

- **`learning_rate`**: Learning rate of the model training.


In [25]:
ai.train(file_name,
         line_by_line=False,
         from_cache=False,
         num_steps=5000,
         generate_every=250,
         save_every=250,
         save_gdrive=True,
         learning_rate=1e-3,
         batch_size=256,
         )

11/26/2021 13:51:16 — INFO — aitextgen — Loading text from _chat.txt with generation length of 64.


  0%|          | 0/59244 [00:00<?, ?it/s]

11/26/2021 13:51:16 — INFO — aitextgen.TokenDataset — Encoding 59,244 sets of tokens from _chat.txt.
11/26/2021 13:51:20 — WARNING — aitextgen — pytorch_model.bin already exists in /trained_model and will be overwritten!
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:148: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the 

  0%|          | 0/5000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1819: LightningDeprecationWarning: `trainer.progress_bar_dict` is deprecated in v1.5 and will be removed in v1.7. Use `ProgressBarBase.get_metrics` instead.
  "`trainer.progress_bar_dict` is deprecated in v1.5 and will be removed in v1.7."


250 steps reached: saving model to /trained_model
250 steps reached: generating sample texts.
 per fatto tu mi ha fatto +00
[25/08/20, 23:02:27] Alessandra Corsi 🐍: Ma che va bene
[25/08/20, 23:02:32] Alessandra Corsi 🐍: Non mi dicevano
[25/08/20, 23:
500 steps reached: saving model to /trained_model
500 steps reached: generating sample texts.
!
‎[19/11/20, 17:29:08] Filippo: ‎audio omitted
[19/11/20, 17:29:22] Filippo: Baby
[19/11/20, 17:29:37] Filippo: Non so l’è sicuro
[
750 steps reached: saving model to /trained_model
750 steps reached: generating sample texts.
 senza
[14/11/19, 21:36:25] Filippo: Vedrai che sei tuzzo dice
[14/11/19, 21:36:30] Filippo: Ahahhahahhahahahahaah
[14/11/19, 21:36
1,000 steps reached: saving model to /trained_model
1,000 steps reached: generating sample texts.
 e poi attenzione
[10/11/20, 12:33:14] Alessandra Corsi 🐍: Io ho impazzito e ti devo rimani
[10/11/20, 12:33:16] Filippo: Baby
[10/11/20, 12
1,250 steps reached: saving model to /trained_model
1,25

11/26/2021 14:25:51 — INFO — aitextgen — Saving trained model pytorch_model.bin to /trained_model


You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.


## Load a Trained Model

Running the next cell will copy the `pytorch_model.bin`, `config.json`, `aitextgen_vocab.json`, and `aitextgen_merges.json` files from the specified folder in Google Drive into the Colaboratory VM. (If no `from_folder` is specified, it assumes the two files are located at the root level of your Google Drive)

In [33]:
from_folder = None

for file in ["pytorch_model.bin", "config.json", "aitextgen.tokenizer.json"]:
  if from_folder:
    copy_file_from_gdrive(file, from_folder)
  else:
    copy_file_from_gdrive(file)

The next cell will allow you to load the retrained model + metadata necessary to generate text.

In [34]:
ai = aitextgen(model_folder=".",
               tokenizer_file="aitextgen.tokenizer.json",
               to_gpu=True)

11/26/2021 14:32:38 — INFO — aitextgen — Loading model from provided weights and config in /..
11/26/2021 14:32:38 — INFO — aitextgen — GPT2 loaded with 1M parameters.
11/26/2021 14:32:38 — INFO — aitextgen — Using a custom tokenizer.


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text.

**If you just trained a model**, you'll get much faster training performance if you reload the model; the next cell will reload the model you just trained from the `trained_model` folder.

In [35]:
ai = aitextgen(model_folder="trained_model",
               tokenizer_file="aitextgen.tokenizer.json",
               to_gpu=True)

11/26/2021 14:32:50 — INFO — aitextgen — Loading model from provided weights and config in /trained_model.
11/26/2021 14:32:50 — INFO — aitextgen — GPT2 loaded with 1M parameters.
11/26/2021 14:32:50 — INFO — aitextgen — Using a custom tokenizer.


`generate()` without any parameters generates a single text from the loaded model to the console.

In [36]:
ai.generate()

baby
[24/01/19, 16:40:36] Alessandra Corsi 🐍: 🤩
[24/01/19, 17:11:12] Alessandra Corsi 🐍: Yes
[24/01/19, 17:11:17] Alessandra Corsi 🐍: 👻🎣


If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = ai.generate_one()`

You can also pass in a `prompt` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `n`. You can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 50 for `batch_size` to avoid going OOM).

Other optional-but-helpful parameters for `ai.generate()` and friends:

*  **`max_length`**: Number of tokens to generate (default 256, you can generate up to 1024 tokens with GPT-2, but it will be _much_ slower)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)

In [45]:
ai.generate(max_length = 512,
            n=5,
            batch_size=5,
            prompt="[24/01/19, 16:40:36] Alessandra Corsi 🐍:",
            temperature=0.95,
            top_p=0.95)

[24/01/19, 16:40:36] Alessandra Corsi 🐍: Sono contenta!!!!!
[24/01/19, 17:10:53] Alessandra Corsi 🐍: Sei un bu che ho squindi il mio ononomasticinsa
[24/01
[24/01/19, 16:40:36] Alessandra Corsi 🐍: Ecco
[24/01/19, 16:40:47] Alessandra Corsi 🐍: Non riesco ad alzarmi
[24/01/19, 16:42:20] Filippo: Best a
[24/01/19, 16:40:36] Alessandra Corsi 🐍: Che bello!
[24/01/19, 16:41:14] Alessandra Corsi 🐍: Io mi asciugo i capelliiiiz
[24/01/19, 16:41:19]
[24/01/19, 16:40:36] Alessandra Corsi 🐍: Sei un cuore
[24/01/19, 20:02:46] Filippo: Yes
[24/01/19, 20:02:44] Filippo: Bzzz
[24/01/
[24/01/19, 16:40:36] Alessandra Corsi 🐍: Ma tipo?
[24/01/19, 16:40:38] Alessandra Corsi 🐍: O no
[24/01/19, 16:40:41] Alessandra Corsi 🐍: Okii
[24/01/


For bulk generation, you can generate a large amount of texts to a file and sort out the samples locally on your computer. The next cell will generate `num_files` files, each with `n` texts and whatever other parameters you would pass to `generate()`. The files can then be downloaded from the Files sidebar!

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
num_files = 5

for _ in range(num_files):
  ai.generate_to_file(n=1000,
                     batch_size=50,
                     prompt="ROMEO:",
                     temperature=1.0,
                     top_p=0.9)

04/19/2021 01:59:34 — INFO — aitextgen — Generating 1,000 texts to ATG_20210419_015934_63053824.txt


04/19/2021 02:00:04 — INFO — aitextgen — Generating 1,000 texts to ATG_20210419_020004_96905312.txt


04/19/2021 02:00:33 — INFO — aitextgen — Generating 1,000 texts to ATG_20210419_020033_47861406.txt


04/19/2021 02:01:03 — INFO — aitextgen — Generating 1,000 texts to ATG_20210419_020103_74636692.txt


04/19/2021 02:01:33 — INFO — aitextgen — Generating 1,000 texts to ATG_20210419_020133_64281888.txt


# LICENSE

MIT License

Copyright (c) 2020-2021 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.